## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-01-10-52 +0000,nyt,Fired Scholars and Big Grants to Favored Proje...,https://www.nytimes.com/2025/11/15/arts/nation...
1,2025-11-17-01-07-02 +0000,nypost,United Airlines flight makes emergency landing...,https://nypost.com/2025/11/16/us-news/united-a...
2,2025-11-17-01-04-52 +0000,48hills,Roddy Bottum: ‘It felt like a sense of royalty...,https://48hills.org/2025/11/roddy-bottum-it-fe...
3,2025-11-17-01-02-59 +0000,nyt,"At Catholic Churches, Trump’s Immigration Crac...",https://www.nytimes.com/2025/11/16/us/catholic...
4,2025-11-17-01-00-00 +0000,wsj,Regulators Bypassed Building 602. Then It Blew...,https://www.wsj.com/business/regulators-bypass...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,20
387,epstein,9
173,new,8
56,killed,7
43,up,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...,58
107,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,54
141,2025-11-16-08-12-45 +0000,bbc,Feud erupts between Trump and ally Marjorie Ta...,https://www.bbc.com/news/articles/c9wv4dx05q5o...,54
52,2025-11-16-21-32-28 +0000,bbc,Marjorie Taylor Greene doubles down on Epstein...,https://www.bbc.com/news/articles/cdeg8z0j2j2o...,53
114,2025-11-16-14-50-10 +0000,nypost,Marjorie Taylor Greene insists she still suppo...,https://nypost.com/2025/11/16/us-news/marjorie...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,58,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...
32,32,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
20,28,2025-11-17-00-00-21 +0000,bbc,Formula 1 tech used to rev-up train wi-fi spee...,https://www.bbc.com/news/articles/cqjwkdez71ro...
6,27,2025-11-17-00-51-22 +0000,nypost,22-year-old ID’d as gunman killed in shootout ...,https://nypost.com/2025/11/16/us-news/gunman-k...
74,27,2025-11-16-20-23-31 +0000,nypost,Jets cornerback Kris Boyd’s pal gives update o...,https://nypost.com/2025/11/16/sports/jets-corn...
13,26,2025-11-17-00-19-40 +0000,nypost,DSA warns Gov. Kathy Hochul over ‘third strike...,https://nypost.com/2025/11/16/us-news/liberals...
41,26,2025-11-16-22-45-28 +0000,nypost,Canal Street knock-off vendors back with venge...,https://nypost.com/2025/11/16/us-news/nyc-knoc...
119,25,2025-11-16-13-33-18 +0000,bbc,The US will help South Korea build nuclear 'at...,https://www.bbc.com/news/articles/c620qppzlgwo...
148,24,2025-11-16-05-06-13 +0000,nyt,4 Dead After Suspected Migrant Boat Overturns ...,https://www.nytimes.com/2025/11/15/us/panga-bo...
106,24,2025-11-16-17-00-00 +0000,wsj,Over 60% of Americans are comfortable with how...,https://www.wsj.com/politics/policy/economic-o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
